In [41]:
import galois
import numpy as np
import math
# from py_ecc.bn128 import curve_order

curve_order = 967

# Creating a finite field (Galois field) which is basically Fq where q = 967
GF967 = galois.GF(curve_order)
# print(GF.properties)

######################################

def process_array(array):
    (rows, columns) = array.shape
    for i in range(0, rows):
        for j in range(0, columns):
            if(math.modf(array[i][j])[0] == 0.0):
                if(array[i][j] < 0):
                    array[i][j] = GF967(0) - GF967(-(array[i][j]))
            else:
                raise Exception("Fractional values not allowed in the R1CS arrays over prime field", GF967)
    return GF967(array)

######################################

lagrange_x_values = GF967(np.array([1,2,3]))

# These are the default R1CS values of L, R, S and W.
# Later we can take user inputs to create custom R1CS
default_L = GF967(np.array([
    [0,0,1,0,0,0],
    [0,0,0,1,0,0],
    [0,0,0,0,0,4]
]))

default_R = GF967(np.array([
    [0,0,1,0,0,0],
    [0,0,0,1,0,0],
    [0,0,1,0,0,0]
]))

S = np.array([
    [0,0,0,0,1,0],
    [0,0,0,0,0,1],
    [2,1,0,0,-1,0]
])

default_S = process_array(S)

default_W = GF967(np.array([1,199,3,4,9,16]))

###############################################

def reverse_tuple(tuple):
    new_tuple = tuple[::-1]
    return new_tuple

def find_num_columns_in_array(arr):
    shape_tuple = arr.shape
    if(len(shape_tuple) == 2): 
        return shape_tuple[1]
    elif (len(shape_tuple) == 1): 
        return shape_tuple[0]
    else:
        raise Exception("Invalid array passed")
    
def find_num_points_for_lagrange(arr):
    shape_tuple = arr.shape
    return shape_tuple[0]

def convert_column_to_lagrange_polynomial(array, col_num: int):
    required_column = array[ :, col_num: (col_num + 1)].flatten()
    col_polynomial = galois.lagrange_poly(lagrange_x_values, required_column)
    
    if(col_polynomial == 0):
        col_polynomial_array = GF967(np.zeros(find_num_points_for_lagrange(array), dtype = int))
    elif(col_polynomial.coeffs.size < find_num_points_for_lagrange(array)):
        num_zeros_required = find_num_points_for_lagrange(array) - np.asarray(col_polynomial.coeffs).shape[0]
        col_polynomial_array = GF967(np.append(np.zeros(num_zeros_required, dtype=int), (np.asarray(col_polynomial.coeffs))))
    else:
        col_polynomial_array = GF967(np.asarray(col_polynomial.coeffs))
    
    return col_polynomial_array

def convert_arrays_into_poly_arrays(array):
    num_columns = find_num_columns_in_array(array)
    for i in range(num_columns):
        if(i == 0):
            first_arr = convert_column_to_lagrange_polynomial(array, 0)
            poly_arrays = GF967(np.array([first_arr]))
        else:
            poly_arr = convert_column_to_lagrange_polynomial(array, i)
            poly_arrays = GF967(np.append(poly_arrays, poly_arr))

    required_poly_array_shape = reverse_tuple(array.shape)
    return GF967(poly_arrays.reshape(required_poly_array_shape).transpose())

def multiply_poly_arrays_with_witness(array):
    return np.matmul(array, default_W)

# ###############################################

U = convert_arrays_into_poly_arrays(default_L)
# print(U)

# print("***********")

V = convert_arrays_into_poly_arrays(default_R)
# print(V)

# print("***********")

W = convert_arrays_into_poly_arrays(default_S)
# print(W)

# print("***********")

###############################################

Ua = multiply_poly_arrays_with_witness(U)
Va = multiply_poly_arrays_with_witness(V)
Wa = multiply_poly_arrays_with_witness(W)

# print(Ua)
# print("***********")
# print(Va)
# print("***********")
# print(Wa)

###############################################
## Introducing t(x) and h(x)

def negateGF967(num):
    return GF967(0) - GF967(num)

a = galois.Poly(Ua, GF967)
b = galois.Poly(Va, GF967)
c = galois.Poly(Wa, GF967)
t = galois.Poly(GF967(np.array([1, negateGF967(1)])), GF967) * galois.Poly(GF967(np.array([1, negateGF967(2)])), GF967) * galois.Poly(GF967(np.array([1, negateGF967(3)])), GF967)

print("a: ", a)
print("b: ", b)
print("c: ", c)
print("t: ", t)

h = ((a * b) - c ) // t

print("h: ", h) 

###############################################

a:  513x^2 + 396x + 61
b:  966x^2 + 4x
c:  568x^2 + 237x + 171
t:  x^3 + 961x^2 + 11x + 961
h:  454x + 512


In [22]:
x = 53.5123
print(math.modf(53.5123))
print(math.modf(x))
print(math.modf(x)[0] == 0.0)
print(math.modf(x)[0] * 100)
x = math.modf(x)[0] * 100
print(int(x))

example_1 = -6.5 % 11
print(example_1) # 3.5
# Since, 3.5 == 35 x 10**-1
print((35%11 * pow(10, -1, 11))%11)

example_2 = (65%11 * pow(10, -1, 11))%11
print(-example_2 % 11)

## example 3
## -6 in GF (the above defined galios field)
print(GF967(0) - GF967(6))
print(GF967(0) - GF967(1))

## example 4
## 5/2 in GF (the above defined galios field)
print(GF967(0) - (GF967(5) * GF967(2)**-1))

(0.5123000000000033, 53.0)
(0.5123000000000033, 53.0)
False
51.23000000000033
51
4.5
9
10
961
966
481
